In [4]:
# Cell 1
import sys, pickle, os
sys.path.append('../src')

from utils import load_words
from hmm_model import HangmanHMM
from hangman_env import HangmanEnv
from rl_agent import QLearningAgent

# Cell 2
print("Loading models...")
with open('../models/hmm_model.pkl', 'rb') as f:
    hmm = pickle.load(f)
print("✓ HMM loaded | corpus:", len(hmm.corpus_words))
print("Has _smart_fallback:", hasattr(hmm, '_smart_fallback'))

agent = QLearningAgent(hmm)

test_words = load_words('../data/test.txt')

# Cell 3
print("\nEvaluating...")
env = HangmanEnv(test_words)
wins = 0
total_wrong = 0
for i, word in enumerate(test_words):
    env.target_word = word
    env.guessed_letters = set()
    env.wrong_guesses = 0
    env.game_over = False
    env.won = False

    state = env._get_state()
    done = False
    while not done:
        probs = hmm.predict_probabilities(state['masked_word'], state['guessed_letters'])
        action = agent.choose_action(state, probs)
        if action is None:
            break
        state, reward, done, info = env.step(action)

    wins += 1 if env.won else 0
    total_wrong += env.wrong_guesses
    if (i+1) % 500 == 0:
        print(f"Progress: {i+1}/{len(test_words)}")

# Cell 4
success_rate = wins / len(test_words)
score = (success_rate * 2000) - (total_wrong * 5)

print("\n========== FINAL ==========")
print("Games won:", wins, "/", len(test_words))
print(f"Success rate: {success_rate:.1%}")
print("Total wrong guesses:", total_wrong)
print("Score:", int(score))

os.makedirs('../results', exist_ok=True)
with open('../results/final_score.txt', 'w') as f:
    f.write(f"Success: {success_rate:.1%}\n")
    f.write(f"Score: {int(score)}\n")
    f.write(f"Total wrong: {total_wrong}\n")
print("✓ Saved ../results/final_score.txt")


Loading models...
✓ HMM loaded | corpus: 48446
Has _smart_fallback: True
Loaded 2000 words

Evaluating...
Progress: 500/2000
Progress: 1000/2000
Progress: 1500/2000
Progress: 2000/2000

========== FINAL ==========
Games won: 282 / 2000
Success rate: 14.1%
Total wrong guesses: 11394
Score: -56688
✓ Saved ../results/final_score.txt
